In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from datetime import datetime
import time
from torch.utils.tensorboard import SummaryWriter

In [2]:
print(torch.__version__)

2.1.1+cu118


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

In [6]:
train_set = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('../data', train=False, transform=transform)

In [7]:
def ConvBlock(first_chanels, second_chanels): # (8, 16)
    return nn.Sequential(
            nn.Conv2d(second_chanels, first_chanels, 1), #  16, 8
            nn.BatchNorm2d(first_chanels),  # 8
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(first_chanels, second_chanels, 3, padding=1), # 8 16
            nn.BatchNorm2d(second_chanels), 
            nn.LeakyReLU(0.1, inplace=True), # (64, 16, w, h)
        )

In [8]:
class Net(nn.Module):
    def __init__(self, num_classes, drop_rate):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.drop_rate = drop_rate
        
        self.conv1 = nn.Conv2d(1, 16, 5, stride=2, padding=2) # (batch, 16, 14, 14)
        self.lrelu1 = nn.LeakyReLU(0.1, inplace=True)
        self.bn1 = nn.BatchNorm2d(16)
        self.convb1 = ConvBlock(8, 16) # (batch, 16, 7, 7)
        self.maxp1 = nn.MaxPool2d(2)  # (batch, 16, 7, 7)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # (batch, 32, 7, 7)
        self.lrelu2 = nn.LeakyReLU(0.1, inplace=True)
        self.bn2 = nn.BatchNorm2d(32)
        self.convb2 = ConvBlock(16, 32)
        self.maxp2 = nn.MaxPool2d(2) # (batch, 32, 4, 4)

        self.fc1 = nn.LazyLinear(1000)
        self.relu = nn.ReLU(1000)
        self.dropout = nn.Dropout(self.drop_rate)
        self.fc2 = nn.Linear(1000, self.num_classes)
        
        
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)


    def forward(self, x):
        batch, _, _, _ = x.size()
        x = self.conv1(x)
        x = self.lrelu1(x)
        x = self.bn1(x)
        x = self.convb1(x)
        x = self.maxp1(x)

        x = self.conv2(x)
        x = self.lrelu2(x)
        x = self.bn2(x)
        x = self.convb2(x)
        x = self.maxp2(x)

        x = x.view(batch, -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# train func

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 300 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    return loss.item()

In [10]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss, 100. * correct / len(test_loader.dataset) 

# model init

In [11]:
model = Net(10, 0.1)

C:\Users\da4nik\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [12]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=0)

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

In [14]:
model.to(device)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (lrelu1): LeakyReLU(negative_slope=0.1, inplace=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convb1): Sequential(
    (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (lrelu2): LeakyReLU(negative_slope=0.1, inplace=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (convb2): Sequential(

# train

In [15]:
model_name = "model1"

In [16]:
scheduler = StepLR(optimizer, step_size=1)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/' + model_name + '_{}'.format(timestamp))

for epoch in range(1, 200 + 1):
    start_epoch = time.time()
    train_loss = train(model, device, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, device, test_loader)
    scheduler.step()
    end_epoch = time.time()
    elapsed = end_epoch - start_epoch
    print("Time per epoch {}s".format(elapsed))

    writer.add_scalars('Training vs. Test Loss',
                    { 'Training' : train_loss, 'Test' : test_loss },
                    epoch + 1)
    writer.add_scalars('Test Accuracy',
                    { 'Test' : test_loss },
                    epoch + 1)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.288904
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.087876
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.116637
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.028818

Test set: Average loss: 0.0805, Accuracy: 9739/10000 (97%)

Time per epoch 18.606622457504272s
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.022370
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.047456
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.022563
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.016510

Test set: Average loss: 0.0277, Accuracy: 9913/10000 (99%)

Time per epoch 15.862976551055908s
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.067525
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.048298
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.032998
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.031820

Test set: Average loss: 0.0262, Accuracy: 9912/10000 (99%)

Time per epoch 15.58545184135437s
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.006728
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.050909
Train Epoch: 4 [38400/60000 (64%)

In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%reload_ext tensorboard

In [4]:
%tensorboard --logdir runs/ --port=6009